### This notebook is to prepare data for analysis
#### Group Project on 'Public_Health' (Focus: Substance abuse in young adults)
#### Members: Jamuna, Eddie, Warren, Anna
#### Start Date: 05/15/2019

In [2]:
#Dependencies
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes as ax
from scipy import stats
import seaborn as sns

In [7]:
# Explore input data from the csvs
adm_csvdata = pd.read_csv("admissions.csv")
dis_csvdata = pd.read_csv("discharges.csv")
print(f"Total number of records in admissions.csv is : %d" %len(adm_csvdata))
print(f"Total number of records in discharges.csv is : %d" %len(dis_csvdata))
#csvdata.columns

Total number of records in admissions.csv is : 1048575
Total number of records in discharges.csv is : 1048575


In [21]:
# Apply filters to select records of interest for analysis (rows filtered based on Primary substance - SUB1)
adm_data = adm_csvdata[adm_csvdata['SUB1'].isin([1,2,3,4,5,7,13,19]) & adm_csvdata['AGE'].isin([2,3,4]) ]
dis_data = dis_csvdata[dis_csvdata['SUB1'].isin([1,2,3,4,5,7,13,19]) & dis_csvdata['AGE'].isin([2,3,4])]
#adm_csvdata[adm_csvdata['SUB1'].isin[1,2,3,4,5,7,13,19]]
print(f"Total number of records filtered from admissions.csv is : %d" %len(adm_data))
print(f"Total number of records filtered from discharges.csv is : %d" %len(dis_data))

Total number of records filtered from admissions.csv is : 160112
Total number of records filtered from discharges.csv is : 164370


In [23]:
# Apply further filters on columns of interest based on the lookup document for features coded in the csvs
# List the columns from admissions and discharge datasets
adm_data.columns

Index(['CASEID', 'STFIPS (Census state FIPS code)', 'AGE', 'SERVICES', 'SUB1',
       'SUB2', 'SUB3', 'DETCRIM', 'NOPRIOR', 'PSOURCE', 'ARRESTS', 'RACE',
       'ETHNIC', 'SEX', 'EDUC', 'EMPLOY', 'METHUSE', 'PSYPROB', 'PREG', 'VET',
       'LIVARAG', 'PRIMINC', 'HLTHINS', 'PRIMPAY', 'DETNLF', 'MARSTAT',
       'DAYWAIT', 'ROUTE1', 'FREQ1', 'FRSTUSE1', 'ROUTE2', 'FREQ2', 'FRSTUSE2',
       'ROUTE3', 'FREQ3', 'FRSTUSE3', 'FREQ_ATND_SELF_HELP', 'DSMCRIT',
       'ALCFLG', 'COKEFLG', 'MARFLG', 'HERFLG', 'METHFLG', 'OPSYNFLG',
       'PCPFLG', 'HALLFLG', 'MTHAMFLG', 'AMPHFLG', 'STIMFLG', 'BENZFLG',
       'TRNQFLG', 'BARBFLG', 'SEDHPFLG', 'INHFLG', 'OTCFLG', 'OTHERFLG',
       'DIVISION', 'REGION', 'ALCDRUG', 'YEAR', 'CBSA10', 'IDU'],
      dtype='object')

In [25]:
dis_data.columns

Index(['DISYR', 'CASEID', 'STFIPS', 'AGE', 'SERVICES', 'SUB1', 'SUB2', 'SUB3',
       'DETCRIM', 'NOPRIOR', 'PSOURCE', 'ARRESTS', 'RACE', 'ETHNIC', 'EDUC',
       'EMPLOY', 'METHUSE', 'PSYPROB', 'PREG', 'VET', 'LIVARAG', 'PRIMINC',
       'HLTHINS', 'PRIMPAY', 'DETNLF', 'MARSTAT', 'DAYWAIT', 'ROUTE1', 'FREQ1',
       'FRSTUSE1', 'ROUTE2', 'FREQ2', 'FRSTUSE2', 'ROUTE3', 'FREQ3',
       'FRSTUSE3', 'FREQ_ATND_SELF_HELP', 'DSMCRIT', 'SERVICES_D', 'REASON',
       'SUB1_D', 'SUB2_D', 'SUB3_D', 'EMPLOY_D', 'LIVARAG_D', 'DETNLF_D',
       'FREQ1_D', 'FREQ2_D', 'FREQ3_D', 'FREQ_ATND_SELF_HELP_D', 'LOS',
       'ARRESTS_D', 'ALCFLG', 'COKEFLG', 'MARFLG', 'HERFLG', 'METHFLG',
       'OPSYNFLG', 'PCPFLG', 'HALLFLG', 'MTHAMFLG', 'AMPHFLG', 'STIMFLG',
       'BENZFLG', 'TRNQFLG', 'BARBFLG', 'SEDHPFLG', 'INHFLG', 'OTCFLG',
       'OTHERFLG', 'NUMSUBS', 'IDU', 'DIVISION', 'REGION', 'ALCDRUG', 'YEAR',
       'CBSA', 'GENDER'],
      dtype='object')

In [37]:
# Choose features of interest for further analysis for this project
adm_study_data = adm_data[['CASEID', 'STFIPS (Census state FIPS code)','AGE', 'SERVICES', 'SUB1', 'SUB2', 
       'DETCRIM', 'NOPRIOR', 'PSOURCE', 'SEX', 'EDUC', 'EMPLOY', 'METHUSE', 'PSYPROB',
       'LIVARAG', 'PRIMINC', 'HLTHINS', 'PRIMPAY', 'DETNLF', 'MARSTAT','DAYWAIT', 'ROUTE1', 
       'FREQ1', 'FRSTUSE1', 'ROUTE2', 'FREQ2', 'FRSTUSE2','FREQ_ATND_SELF_HELP', 'DSMCRIT',
       'ALCFLG', 'COKEFLG', 'MARFLG', 'HERFLG', 'OPSYNFLG','HALLFLG', 'BENZFLG','OTHERFLG',
       'REGION', 'ALCDRUG', 'YEAR', 'IDU']]
# Get the dimensions of the dataset
adm_study_data.shape

(160112, 41)

In [39]:
dis_study_data = dis_data[['CASEID', 'STFIPS', 'AGE', 'SERVICES_D','SUB1_D', 'SUB2_D', 
       'DETCRIM', 'NOPRIOR', 'PSOURCE','GENDER','EDUC','EMPLOY', 'METHUSE', 'PSYPROB',
       'LIVARAG', 'PRIMINC', 'HLTHINS', 'PRIMPAY', 'DETNLF', 'MARSTAT', 'DAYWAIT', 'ROUTE1', 
       'FREQ1', 'FRSTUSE1', 'ROUTE2', 'FREQ2', 'FRSTUSE2', 'FREQ_ATND_SELF_HELP', 'DSMCRIT', 
       'REASON', 'EMPLOY_D', 'LIVARAG_D', 'DETNLF_D', 'FREQ1_D', 'FREQ2_D', 
       'FREQ_ATND_SELF_HELP_D', 'LOS','ALCFLG', 'COKEFLG', 'MARFLG', 'HERFLG', 'OPSYNFLG', 
       'HALLFLG', 'BENZFLG','OTHERFLG', 'NUMSUBS', 'IDU', 'REGION', 'ALCDRUG', 'DISYR']]
# Get the dimensions of the dataset
dis_study_data.shape

(164370, 50)

In [40]:
# Rename some of the columns to merge dataframes without conflicts
#df1.merge(df2.rename(columns={'b':'a'}),how='outer')
adm_study_data = adm_study_data.rename(columns={'STFIPS (Census state FIPS code)':'STFIPS',
                                                'SEX': 'GENDER'})

In [57]:
# Merge the two datasets based on 'CASEID'
merged_sub_abuse_data = pd.merge(adm_study_data, dis_study_data, on='CASEID', how='outer', suffixes=('_adm', '_dis') )
merged_sub_abuse_data.head()

,CASEID,STFIPS_adm,AGE_adm,SERVICES,SUB1,SUB2,DETCRIM_adm,NOPRIOR_adm,PSOURCE_adm,GENDER_adm,...,HERFLG_dis,OPSYNFLG_dis,HALLFLG_dis,BENZFLG_dis,OTHERFLG_dis,NUMSUBS,IDU_dis,REGION_dis,ALCDRUG_dis,YEAR_dis
0,20161570407,2.0,3.0,7.0,2.0,1.0,-9.0,1.0,6.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20161487219,2.0,3.0,7.0,2.0,1.0,6.0,3.0,7.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20161299121,2.0,4.0,6.0,7.0,2.0,-9.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20161527363,2.0,4.0,7.0,2.0,-9.0,5.0,0.0,7.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20161383667,2.0,4.0,7.0,2.0,7.0,-9.0,4.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
merged_sub_abuse_data.columns

Index(['CASEID', 'STFIPS_adm', 'AGE_adm', 'SERVICES', 'SUB1', 'SUB2',
       'DETCRIM_adm', 'NOPRIOR_adm', 'PSOURCE_adm', 'GENDER_adm', 'EDUC_adm',
       'EMPLOY_adm', 'METHUSE_adm', 'PSYPROB_adm', 'LIVARAG_adm',
       'PRIMINC_adm', 'HLTHINS_adm', 'PRIMPAY_adm', 'DETNLF_adm',
       'MARSTAT_adm', 'DAYWAIT_adm', 'ROUTE1_adm', 'FREQ1_adm', 'FRSTUSE1_adm',
       'ROUTE2_adm', 'FREQ2_adm', 'FRSTUSE2_adm', 'FREQ_ATND_SELF_HELP_adm',
       'DSMCRIT_adm', 'ALCFLG_adm', 'COKEFLG_adm', 'MARFLG_adm', 'HERFLG_adm',
       'OPSYNFLG_adm', 'HALLFLG_adm', 'BENZFLG_adm', 'OTHERFLG_adm',
       'REGION_adm', 'ALCDRUG_adm', 'YEAR_adm', 'IDU_adm', 'STFIPS_dis',
       'AGE_dis', 'SERVICES_D', 'SUB1_D', 'SUB2_D', 'DETCRIM_dis',
       'NOPRIOR_dis', 'PSOURCE_dis', 'GENDER_dis', 'EDUC_dis', 'EMPLOY_dis',
       'METHUSE_dis', 'PSYPROB_dis', 'LIVARAG_dis', 'PRIMINC_dis',
       'HLTHINS_dis', 'PRIMPAY_dis', 'DETNLF_dis', 'MARSTAT_dis',
       'DAYWAIT_dis', 'ROUTE1_dis', 'FREQ1_dis', 'FRSTUSE1_dis', 

In [53]:
# Drop NaNs (null) from the merged dataset
# This is because, we need cases that have a record of admission with a subsequent discharge. Therefore, drop all the records that
# do not match (comes with a NaN) upon a 'full outer join'.
merged_sub_abuse_data.dropna()

,CASEID,STFIPS_adm,AGE_adm,SERVICES,SUB1,SUB2,DETCRIM_adm,NOPRIOR_adm,PSOURCE_adm,GENDER_adm,...,HERFLG_dis,OPSYNFLG_dis,HALLFLG_dis,BENZFLG_dis,OTHERFLG_dis,NUMSUBS,IDU_dis,REGION_dis,ALCDRUG_dis,YEAR_dis
6,20161308408,2.0,4.0,7.0,2.0,4.0,4.0,1.0,7.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,2.0,2016.0
10,20161318655,2.0,4.0,6.0,2.0,4.0,5.0,1.0,7.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,2016.0
12,20161345139,2.0,3.0,7.0,5.0,10.0,3.0,1.0,7.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,2016.0
15,20161431022,2.0,4.0,7.0,2.0,4.0,3.0,0.0,7.0,1.0,...,1.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,2.0,2016.0
32,20161448923,2.0,4.0,7.0,2.0,4.0,-9.0,0.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,2.0,2016.0
35,20161309135,2.0,3.0,7.0,4.0,9.0,-9.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,3.0,0.0,4.0,3.0,2016.0
37,20161400961,2.0,2.0,7.0,2.0,1.0,-9.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,2.0,2016.0
48,20161449698,2.0,4.0,7.0,2.0,4.0,-9.0,3.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,3.0,2016.0
69,20161401968,2.0,3.0,5.0,5.0,3.0,-9.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,2.0,2016.0
70,20161361222,2.0,4.0,5.0,7.0,1.0,-9.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,2.0,2016.0


In [56]:
# Drop all rows that have been coded with -9 for missing or invalid data 
merged_sub_abuse_data = merged_sub_abuse_data[~(merged_sub_abuse_data == '-9.0').any(axis=1)]
merged_sub_abuse_data

,CASEID,STFIPS_adm,AGE_adm,SERVICES,SUB1,SUB2,DETCRIM_adm,NOPRIOR_adm,PSOURCE_adm,GENDER_adm,...,HERFLG_dis,OPSYNFLG_dis,HALLFLG_dis,BENZFLG_dis,OTHERFLG_dis,NUMSUBS,IDU_dis,REGION_dis,ALCDRUG_dis,YEAR_dis
3325,20161090014,5.0,4.0,7.0,7.0,4.0,6.0,1.0,7.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3668,20161207658,5.0,2.0,7.0,4.0,7.0,2.0,0.0,7.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189336,20161192313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,3.0,0.0,4.0,3.0,2016.0
190281,20161145627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,2.0,1.0,4.0,2.0,2016.0
190428,20161455048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,3.0,2016.0
190429,20161472845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,3.0,2016.0
190726,20161285078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,3.0,2016.0
192156,20161418572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,3.0,0.0,4.0,3.0,2016.0
192539,20161158455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,3.0,2016.0
258078,20160223292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,3.0,2016.0
